# 📌 Uygulama – 6  
# 🧠 “CIFAR-10 Normalize + ImprovedCNN: Tam Preprocess & Inference Pipeline (C++)”  

Bu uygulama, önceki tüm uygulamaların bir üst seviyesidir.  
İlk kez:

- **CIFAR-10 normalize** değerlerini birebir C++’ta uyguladım  
- Kanal bazlı normalize işlemini OpenCV ile yaptım  
- Dennis-tier preprocess pipeline kurdum  
- Eğittiğim *ImprovedCNN* modelini C++’ta doğru biçimde çalıştırdım  

Bu uygulama artık tamamen **production kalite bir model çalıştırma hattıdır.**

---

## 🎯 1) Amaç — *Bu projede ne yaptım?*

Bu uygulamanın amacı:

> **Python’da ImprovedCNN + CIFAR10 ile eğittiğim modeli,  
> aynı normalize değerlerini kullanarak C++ tarafında doğru şekilde çalıştırmak.**

Burada dikkat edilmesi gereken en kritik kısım:

✔ Python eğitim sürecindeki **CIFAR10 MEAN / STD normalize** işlemini  
✔ C++ preprocess içinde **Aynı şekilde uygulamak**

Eğer normalize değerleri tutmazsa → Model çöker.  
Bu uygulamada bu sorunu tamamen çözdüm.

---

## 🧩 2) Ana Adımlar — *Kodda yaptığım işlemler*

---

# 🟦 A) CIFAR-10 Normalize değerlerini tanımladım

```cpp
const float CIFAR10_MEAN[3] = {0.4914f, 0.4822f, 0.4465f};
const float CIFAR10_STD[3]  = {0.2470f, 0.2435f, 0.2616f};
```

* Bu değerler PyTorch CIFAR10 dataset ile birebir aynı

* Python eğitimde kullanılan normalize → C++ tarafında da kullanıldı

Bu, model doğruluğu için HAYATİ önem taşır.


## 🟩 B) preprocess_image() fonksiyonu

### 1️⃣ Orijinali kopyaladım (OpenCV güvenliği)
```cpp
img_bgr.copyTo(img);
```

### 2️⃣ Resize
```cpp
cv::resize(img, img, cv::Size(target_size, target_size));
```

### 3️⃣ BGR → RGB
```cpp
cv::cvtColor(img, img, cv::COLOR_BGR2RGB);
```

### 4️⃣ float32 + normalize [0–1]
```cpp
img.convertTo(img_float, CV_32F, 1.0f / 255.0f);
```

### 5️⃣ 🔥 Kanal bazlı normalize (en önemli kısım)
```cpp
std::vector<cv::Mat> channels(3);
cv::split(img_float, channels); // R, G, B

for (int c = 0; c < 3; ++c)
{
    channels[c] = (channels[c] - CIFAR10_MEAN[c]) / CIFAR10_STD[c];
}

cv::merge(channels, img_float);
```


* Artık tensör tamamen Python’daki normalize değerleri ile birebir aynı hale geldi.

### 6️⃣ HWC → Tensor (NHWC formatı)
```cpp
auto tensor_image = torch::from_blob(
    img_float.data,
    {1, img_float.rows, img_float.cols, 3},
    torch::TensorOptions().dtype(torch::kFloat32)
);
```

### 7️⃣ NHWC → NCHW dönüşümü + clone()
```cpp
tensor_image = tensor_image.permute({0, 3, 1, 2});
tensor_image = tensor_image.contiguous().clone();
```


* Final shape: [1, 3, 64, 64]

## 🟥 C) Inference Pipeline – run_inference()
```cpp
std::vector<torch::jit::IValue> inputs;
inputs.push_back(input);

torch::Tensor output = module.forward(inputs).toTensor();
```


Modüler forward

* C++ tarafında profesyonel inference standardı

## 🟦 D) Ana Program (main)
✔ Model dosyasını kontrol ettim

✔ TorchScript modeli yükledim

✔ CPU + eval moduna aldım

✔ test.jpg dosyasını okudum

✔ preprocess_image() ile tensere dönüştürdüm

✔ Shape doğrulaması yaptım

✔ Inference çalıştırdım

✔ Softmax + Argmax ile sınıf tahmini aldım
```cpp
torch::Tensor probs = torch::softmax(output, 1);
torch::Tensor pred_class = probs.argmax(1);
```


Sonuç:
* Tahmin edilen sınıf index’i başarıyla elde edildi.

---

# 🧠 3) Bu projede öğrendiğim temel kavramlar

📌 CIFAR10 normalize işleminin C++ tarafında yeniden oluşturulması

📌 Kanal bazlı normalize (R/G/B ayrı ayrı)

📌 cv::split + cv::merge teknikleri

📌 Preprocess fonksiyonunun profesyonel olarak modülerleşmesi

📌 TorchScript modelinin production-style pipeline’da çalıştırılması

📌 Shape validation mantığı

📌 Softmax + argmax ile sınıf tahmini

📌 contiguous().clone() ile tensör bellek kontrolü

📌 Eğitilen model ile C++ preprocess’inin tam uyumu

Bu uygulama, seni tam bir inference pipeline mühendisi seviyesine getiren uygulamadır.

---

# ⚠️ 4) Dikkat edilmesi gereken kritik noktalar

Normalize değerleri Python ile birebir aynı olmalı

Kanal sırası doğru olmalı (RGB)

NHWC → NCHW dönüşümü yapılmazsa model çalışmaz

clone() unutulursa bellek pointer’ı OpenCV’de kalır → crash

Modelin input boyutu (64×64) kesinlikle kontrol edilmeli

Softmax eksik bırakılırsa output ham logit olur

---

# 🔖 5) Etiketler / Kategori

Advanced Preprocess

CIFAR10 Normalize

ImprovedCNN

Production Pipeline

TorchScript + OpenCV

Softmax/Argmax

---

# 📝 6) Tek Cümlelik Özet

CIFAR10 normalize değerlerini doğru uygulayarak ImprovedCNN modelimi C++ tarafında profesyonel bir preprocess + inference pipeline ile başarıyla çalıştırdım.